# Whats here?
- Possibility to load and prompt the gemma2 model directly without HOOKS
- Most Gemma 2 up to 9B pars work on RTX3070 -- time taken, see below
- Loading different datasets 
- Ask multiple Qs from Datasets
- write answers to .csv (buggy)


# Setup
Conda Env == lets_drift_env_V1 
(Which is a clone of arena_env1_1_1c)

## Imports for Gemma Model

In [1]:
import torch as t


In [2]:
device = t.device('mps' if t.backends.mps.is_available() else 'cuda' if t.cuda.is_available() else 'cpu')


In [3]:
from huggingface_hub import login
login()  # This will prompt for your HuggingFace token in a text box


starts with a j..jeje .. f_JTQGGDeycTxPCozthrzqWQhmLpOkaVKXRq

In [4]:
""" # Clear any loaded models from RAM ~~ alternatively restart the kernel
import gc


if 'model' in locals() or 'model' in globals():
    del model
if 'tokenizer' in locals() or 'tokenizer' in globals():
    del tokenizer
    
gc.collect()
t.cuda.empty_cache() if t.cuda.is_available() else None """


" # Clear any loaded models from RAM ~~ alternatively restart the kernel\nimport gc\n\n\nif 'model' in locals() or 'model' in globals():\n    del model\nif 'tokenizer' in locals() or 'tokenizer' in globals():\n    del tokenizer\n    \ngc.collect()\nt.cuda.empty_cache() if t.cuda.is_available() else None "

In [16]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
model2use = "gemma2_9b_it"#"gemma2_2b_it"

"""
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", device_map="cuda")



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

gemma2_9b_it loaded


Do a **quick test** that the model is working with a random prompt. 

In [6]:

input_text = "Once upon a time, "
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=64)
print(tokenizer.decode(outputs[0]))

<bos>Once upon a time, 
in a land where the sun always shone,
lived a little girl named Lily.

Lily had hair like spun gold,
eyes as blue as the summer sky,
and a smile that could light up the darkest night.

But Lily was lonely.

She had no one to play with,
no one to share her secrets with,
no one to laugh with.

One day, while wandering through the forest,
Lily stumbled upon a tiny, shimmering door.

It was no bigger than her hand,
and it was made of the most beautiful crystal.

Curiosity got the better of her,



how to make history based contextualization .... (ie use past prompts and answers in it's new answer..)

https://jaswanth04.medium.com/history-based-contextualization-in-llm-deployed-as-a-backend-service-fastapi-and-5a1759fdd32e

In [7]:
"""
input_text = "What did I last ask you?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))"""

'\ninput_text = "What did I last ask you?"\ninput_ids = tokenizer(input_text, return_tensors="pt").to("cuda")\n\noutputs = model.generate(**input_ids, max_new_tokens=128)\nprint(tokenizer.decode(outputs[0]))'

In [17]:
ans= tokenizer.decode(outputs[0])



In [9]:
ans

'<bos>Once upon a time, \nin a land where the sun always shone,\nlived a little girl named Lily.\n\nLily had hair like spun gold,\neyes as blue as the summer sky,\nand a smile that could light up the darkest night.\n\nBut Lily was lonely.\n\nShe had no one to play with,\nno one to share her secrets with,\nno one to laugh with.\n\nOne day, while wandering through the forest,\nLily stumbled upon a tiny, shimmering door.\n\nIt was no bigger than her hand,\nand it was made of the most beautiful crystal.\n\nCuriosity got the better of her,\n'

## Imports for Experiment (1)

In [18]:
import pandas as pd
import json
from pathlib import Path

# Experiment (1) Implementation of the test data from Paper: "How is ChatGPT's performance change over time?"

Both Gemma 2 2B and 9B work fine. Only the IT (instruction trained) versions give good answers. Both say stuff that they think is right, but it's not. 


Folder of git: GIT_HowIsChatGPTChanging

In [19]:
"""not used
# Load CSV file with prompts and expected answers from paper
def load_test_data(csv_path):
    return pd.read_csv(csv_path)"""

'not used\n# Load CSV file with prompts and expected answers from paper\ndef load_test_data(csv_path):\n    return pd.read_csv(csv_path)'

In [20]:
# Load the HOTPOTQA evaluation data and extract relevant columns
hotpotqa_df = pd.read_csv("GIT_HowIsChatGPTChanging/LLMDrift/generation/HOTPOTQA_EVAL.csv") ## HotPotQA is supposed to be a dataset for testing LLMs, 
# specifically for testing the reasoning steps. https://arxiv.org/pdf/2312.04511#:~:text=Scott%20Derrickson%20(born%20July%2016,1966)%20is%20an%20American%20filmmaker%20%E2%80%A6&text=Search%20Tool-,Observation%3A%20%E2%80%A6,American%20filmmaker%2C%20actor%2C%20and%20%E2%80%A6
max_queries=2
test_data = hotpotqa_df[['query', 'ref_answer', 'answer']].head(max_queries).copy() # TEST: 5 rows only


In [13]:
#test_data['answer'].iloc[0]
#test_data

In [21]:
max_tokens = 32
# Function to generate response from Gemma
def generate_gemma_response(prompt, model, tokenizer, max_tokens=256):
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=max_tokens)
    return tokenizer.decode(outputs[0])

In [22]:
# Function to run evaluation
def evaluate_gemma(test_data, model, tokenizer):
    results = []
    for idx, row in test_data.iterrows():
        query = row['query']  # adjust column name based on your CSV structure
        ref_answer = row['ref_answer']  # adjust column name based on your CSV structure
        
        actual_response = generate_gemma_response(query, model, tokenizer)
        
        result = {
            'query': query,
            'ref_answer': ref_answer,
            'actual': actual_response,
            'prompt_id': int(idx) # Convert to int for JSON serialization
        }
        print("idx: ",idx, "\n result: ", result, "\n ref_answer: ", ref_answer)
        results.append(result)
        
    return results

# Run evaluation
results = evaluate_gemma(test_data, model, tokenizer)

# Convert results to DataFrame
results_df = pd.DataFrame(results)


# Save results to JSONL
import json
output_path = f"{model2use}_results.jsonl"
with open(output_path, 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')
print(f"Results saved to {output_path}")
"""
# Save results to CSV
output_path = f"{model_name}_results.csv"
results_df.to_csv(output_path, index=False)
print(f"Results saved to {output_path}")"""


IndexError: list index out of range

Above eval script takes 2min 9s for 2 prompts with max tokens == 32

In [13]:
for idx, row in test_data.iterrows():
    print(row['query'], idx )

# Trying a new Dataset, Natural Questions, Google AI [MAYBE come back to it]
HotPotQA is straightforward for usage, so stick with that for now, although the hotpotQA is almost too difficult for SAE Resaearch, it's designed to reason answers based on step-by-step thinking. The following NQ dataset is quite simple; It's Q's asked by google's users and their answers (in long and short format, as well as multiple answers for some Q where multiple answers are applicable)

-->Note; NQ is not that easy to use.. answers are formed by saying 'word #x to #y of this mega text have the answer in short, long, or alternative..'.. Will need some scripting effort!!

Eval script available here, as well as a way to 'unsimplify' the answers in the 4GB dataset
https://github.com/google-research-datasets/natural-questions/blob/master/nq_eval.py
Simple Dataset is downloaded from: https://ai.google.com/research/NaturalQuestions/download 

Trial run: loads first 3 rows (HUGE file 4GB) (takes 1 min)

In [4]:
# ... existing imports ...
import json
import pandas as pd

def load_partial_jsonl(file_path, num_rows=10):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i >= num_rows:
                break
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load first few rows of the Natural Questions dataset
nq_df = load_partial_jsonl('NaturalQuestions_GoogleAI/simplified-nq-train.jsonl', num_rows=3)

In [ ]:
nq_df.head()

In [6]:
# Print all entries from first row
for column in nq_df.columns:
    print(f"\n{column}:")
    print(nq_df.iloc[0][column])


document_text:
Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when to remove this template message ) </Td> </Tr> </Table> 

Yea, so, better go back to the hotpot QA --> :D